## Data Merge Template

In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [23]:
url = 'https://github.com/mhall-simon/ESG-Investing/blob/main/test-data/AR_raw.xlsx?raw=true'

df = pd.read_excel(url, parse_dates = True, index_col = 0) # this sets index to the dates

df

,"Amazon.com, Inc. (NasdaqGS:AMZN) - Accounts Receivable","Conagra Brands, Inc. (NYSE:CAG) - Accounts Receivable",BRF S.A. (BOVESPA:BRFS3) - Accounts Receivable,Sysco Corporation (NYSE:SYY) - Accounts Receivable,"Honda Motor Co., Ltd. (TSE:7267) - Accounts Receivable",Target Corporation (NYSE:TGT) - Accounts Receivable,The Estée Lauder Companies Inc. (NYSE:EL) - Accounts Receivable,Sony Corporation (TSE:6758) - Accounts Receivable,McDonald's Corporation (NYSE:MCD) - Accounts Receivable,"Coca-Cola FEMSA, S.A.B. de C.V. (NYSE:KOF) - Accounts Receivable",Las Vegas Sands Corp. (NYSE:LVS) - Accounts Receivable,"Fomento Económico Mexicano, S.A.B. de C.V. (BMV:FEMSA UBD) - Accounts Receivable",Unilever PLC (LSE:ULVR) - Accounts Receivable,"Marriott International, Inc. (NasdaqGS:MAR) - Accounts Receivable","PepsiCo, Inc. (NasdaqGS:PEP) - Accounts Receivable",Booking Holdings Inc. (NasdaqGS:BKNG) - Accounts Receivable,Starbucks Corporation (NasdaqGS:SBUX) - Accounts Receivable,Costco Wholesale Corporation (NasdaqGS:COST) - Accounts Receivable,Archer-Daniels-Midland Company (NYSE:ADM) - Accounts Receivable,Campbell Soup Company (NYSE:CPB) - Accounts Receivable,"Best Buy Co., Inc. (NYSE:BBY) - Accounts Receivable",Carnival Corporation & plc (NYSE:CCL) - Accounts Receivable,"The Gap, Inc. (NYSE:GPS) - Accounts Receivable",The Coca-Cola Company (NYSE:KO) - Accounts Receivable,eBay Inc. (NasdaqGS:EBAY) - Accounts Receivable,The Procter & Gamble Company (NYSE:PG) - Accounts Receivable,Kohl's Corporation (NYSE:KSS) - Accounts Receivable,"Walgreens Boots Alliance, Inc. (NasdaqGS:WBA) - Accounts Receivable",Ford Motor Company (NYSE:F) - Accounts Receivable,"Lowe's Companies, Inc. (NYSE:LOW) - Accounts Receivable",Colgate-Palmolive Company (NYSE:CL) - Accounts Receivable,"General Mills, Inc. (NYSE:GIS) - Accounts Receivable",Kimberly-Clark Corporation (NYSE:KMB) - Accounts Receivable,British American Tobacco p.l.c. (LSE:BATS) - Accounts Receivable,Diageo plc (LSE:DGE) - Accounts Receivable,"Yum! Brands, Inc. (NYSE:YUM) - Accounts Receivable",Philip Morris International Inc. (NYSE:PM) - Accounts Receivable,"The Home Depot, Inc. (NYSE:HD) - Accounts Receivable",Kellogg Company (NYSE:K) - Accounts Receivable,The Kroger Co. (NYSE:KR) - Accounts Receivable,...,"Cavco Industries, Inc. (NasdaqGS:CVCO) - Accounts Receivable",Farmer Bros. Co. (NasdaqGS:FARM) - Accounts Receivable,"Movado Group, Inc. (NYSE:MOV) - Accounts Receivable","Lithia Motors, Inc. (NYSE:LAD) - Accounts Receivable","Citi Trends, Inc. (NasdaqGS:CTRN) - Accounts Receivable","MarineMax, Inc. (NYSE:HZO) - Accounts Receivable","PetMed Express, Inc. (NasdaqGS:PETS) - Accounts Receivable",Origin Agritech Limited (NasdaqCM:SEED) - Accounts Receivable,Big 5 Sporting Goods Corporation (NasdaqGS:BGFV) - Accounts Receivable,StoneMor Inc. (NYSE:STON) - Accounts Receivable,BrasilAgro - Companhia Brasileira de Propriedades Agrícolas (BOVESPA:AGRO3) - Accounts Receivable,China XD Plastics Company Limited (NasdaqGM:CXDC) - Accounts Receivable,"Nature's Sunshine Products, Inc. (NasdaqCM:NATR) - Accounts Receivable",Natuzzi S.p.A. (NYSE:NTZ) - Accounts Receivable,Stamps.com Inc. (NasdaqGS:STMP) - Accounts Receivable,"M/I Homes, Inc. (NYSE:MHO) - Accounts Receivable","MGP Ingredients, Inc. (NasdaqGS:MGPI) - Accounts Receivable",VOXX International Corporation (NasdaqGS:VOXX) - Accounts Receivable,Landec Corporation (NasdaqGS:LNDC) - Accounts Receivable,"John B. Sanfilippo & Son, Inc. (NasdaqGS:JBSS) - Accounts Receivable","Carrols Restaurant Group, Inc. (NasdaqGS:TAST) - Accounts Receivable","Sturm, Ruger & Company, Inc. (NYSE:RGR) - Accounts Receivable","iMedia Brands, Inc. (NasdaqCM:IMBI) - Accounts Receivable","Conn's, Inc. (NasdaqGS:CONN) - Accounts Receivable",Winmark Corporation (NasdaqGM:WINA) - Accounts Receivable,Veru Inc. (NasdaqCM:VERU) - Accounts Receivable,Gentherm Incorporated (NasdaqGS:THRM) - Accounts Receivable,"CarParts.com, Inc. (NasdaqGS:PRTS) - Accounts Receivable","Culp, Inc. (NYSE

In [24]:
company_name = []
exchange = []
ticker = []


for equity in df.columns:
    beg = equity.find("(")
    end = equity.find(")")
    
#     equity[beg+1:end] # can use the subset notation to select parts of a string
    
    company_name.append(equity[:beg-1])
    
    exchange.append(equity[beg+1:end].split(":")[0])
    ticker.append(equity[beg+1:end].split(":")[1])


In [25]:
df.columns = ticker
df

,AMZN,CAG,BRFS3,SYY,7267,TGT,EL,6758,MCD,KOF,LVS,FEMSA UBD,ULVR,MAR,PEP,BKNG,SBUX,COST,ADM,CPB,BBY,CCL,GPS,KO,EBAY,PG,KSS,WBA,F,LOW,CL,GIS,KMB,BATS,DGE,YUM,PM,HD,K,KR,...,CVCO,FARM,MOV,LAD,CTRN,HZO,PETS,SEED,BGFV,STON,AGRO3,CXDC,NATR,NTZ,STMP,MHO,MGPI,VOXX,LNDC,JBSS,TAST,RGR,IMBI,CONN,WINA,VERU,THRM,PRTS,CULP,HOFT,ODC,BRID,SPB,LUB,SMP,FLWS,MCRI,MPX,DS,LMNR
Dates,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-03-31,1.550000e+08,NaN,2.165530e+08,NaN,NaN,NaN,NaN,NaN,6.863000e+08,1.275025e+09,75759000.0,5.705000e+09,4.616000e+09,9.740000e+08,3.217000e+09,25915000.0,NaN,NaN,NaN,NaN,NaN,332000000.0,NaN,2.115000e+09,270435000.0,NaN,NaN,NaN,3.034000e+09,NaN,1.316200e+09,NaN,1.944200e+09,NaN,NaN,200000000.0,NaN,NaN,9.493000e+08,NaN,...,NaN,NaN,NaN,41932000.0,NaN,NaN,NaN,NaN,5960000.0,25673000.0,NaN,NaN,5927000.0,136700000.0,1886000.0,NaN,NaN,NaN,NaN,NaN,NaN,18988000.0,NaN,NaN,2507800.0,NaN,7066000.0,NaN,NaN,37082000.0,NaN,9920000.0,51216000.0,NaN,198843000.0,NaN,2414332.0,4588000.0,36951000.0,NaN
2005-06-30,1.410000e+08,NaN,3.065200e+08,NaN,NaN,NaN,NaN,NaN,7.565000e+08,1.753000e+09,66386000.0,5.815000e+09,5.370000e+09,8.010000e+08,3.504000e+09,31811000.0,151638000.0,3.491690e+08,NaN,718000000.0,NaN,418000000.0,NaN,2.349000e+09,269641000.0,NaN,NaN,1.483000e+09,2.832000e+09,NaN,1.315400e+09,NaN,1.985400e+09,1.440000e+09,NaN,212000000.0,NaN,NaN,9.388000e+08,NaN,...,NaN,NaN,NaN,51612000.0,NaN,32374000.0,NaN,NaN,5659000.0,26670000.0,NaN,NaN,5507000.0,123600000.0,1889000.0,NaN,NaN,105077000.0,NaN,NaN,NaN,14257000.0,NaN,NaN,2156600.0,2158130.0,5258000.0,NaN,NaN,38277000.0,24681000.0,8241000.0,53759000.0,216000.0,248681000.0,NaN,2860006.0,5451000.0,32217000.0,NaN
2005-09-30,1.680000e+08,1.400000e+09,3.701800e+08,2.242837e+09,NaN,NaN,9.215000e+08,NaN,7.554000e+08,1.769000e+09,53631000.0,4.818000e+09,5.082000e+09,7.730000e+08,3.757000e+09,36200000.0,148982000.0,2.976840e+08,3.991153e+09,539000000.0,NaN,434000000.0,NaN,2.217000e+09,274238000.0,4.396000e+09,NaN,1.515700e+09,3.297000e+09,NaN,1.337200e+09,1.048000e+09,1.980400e+09,NaN,NaN,227000000.0,NaN,NaN,1.026000e+09,NaN,...,NaN,15465000.0,NaN,47801000.0,NaN,40204000.0,NaN,NaN,4165000.0,27053000.0,NaN,NaN,6442000.0,106400000.0,1887000.0,NaN,31922000.0,95787000.0,11657000.0,34905000.0,NaN,19600000.0,NaN,NaN,2066700.0,2183762.0,7177000.0,NaN,26681000.0,38543000.0,23274000.0,8918000.0,12686000.0,252000.0,260877000.0,13263000.0,2985691.0,6120000.0,36320000.0,NaN
2005-12-31,2.590000e+08,1.260800e+09,5.557080e+08,2.284033e+09,4.162070e+11,5.069000e+09,7.766000e+08,1.025362e+12,7.939000e+08,1.935000e+09,84778000.0,3.955000e+09,3.345000e+09,9.530000e+08,2.643000e+09,30043000.0,190762000.0,5.291500e+08,4.102263e+09,473000000.0,3.750000e+08,430000000.0,NaN,2.281000e+09,322788000.0,4.185000e+09,1.389632e+09,1.396300e+09,3.075000e+09,9000000.0,1.309400e+09,8.910000e+08,1.873100e+09,1.093000e+09,1.244000e+09,236000000.0,1.898000e+09,1.499000e+09,7.758000e+08,6.610000e+08,...,7545000.0,15485000.0,NaN,53990000.0,NaN,14570000.0,1796756.0,NaN,7900000.0,29991000.0,NaN,NaN,8313000.0,123608000.0,2131000.0,NaN,28097000.0,128430000.0,15881000.0,39002000.0,3017000.0,15777000.0,79405000.0,9528000.0,3243700.0,2040476.0,7891000.0,1637000.0,28824000.0,43993000.0,23611000.0,9508000.0,NaN,151000.0,176294000.0,10619000.0,3559602.0,3191000.0,38302000.0,2054000.0
2006-03-31,2.140000e+08,1.286800e+09,4.255000e+08,2.406855e+09,7.071500e+11,4.857000e+09,9.152000e+08,9.392810e+11,7.424000e+08,1.978000e+09,108450000.0,5.292454e+09,5.318000e+09,1.001000e+09,3.634000e+09,33821000.0,197765000.0,4.054970e+08,4.309043e+09,788000000.0,3.500000e+08,425000000.0,NaN,2.237000e+09,318878000.0,4.690000e+09,1.309303e+09,1.472800e+09,3.296000e+09,18000000.0,1.320800e+09,1.108000e+09,2.147900e+09,NaN,2.185000e+09,258000000.0,NaN,1.973000e+09,1.028100e+09,6.160000e+08,...,9107000.0,13276000.0,99925000.0,52109000.0,NaN,26515000.0,1912458.0,1937000.0,5484000.0,29805000.0,NaN,NaN,NaN,125400000.0,2303000.0,NaN,32

In [26]:
master = pd.DataFrame(columns={'Dates', 'Ticker'}) # creates an empty multi-index master file
master = master.set_index(['Dates', 'Ticker'])


In [27]:
for col in df.columns:
    idx = pd.MultiIndex.from_product([df.index, [col]], names = ['Dates', 'Ticker'])
    data = pd.DataFrame(df.loc[:,col].values, index = idx)
    data = data.rename(columns={0:"AR"})

    data.dropna(inplace=True)

    master = master.append(data)
    

In [28]:
master

,,AR
Dates,Ticker,
2005-03-31,AMZN,155000000.0
2005-06-30,AMZN,141000000.0
2005-09-30,AMZN,168000000.0
2005-12-31,AMZN,259000000.0
2006-03-31,AMZN,214000000.0
...,...,...
2019-12-31,LMNR,18099000.0
2020-03-31,LMNR,18555000.0
2020-12-31,LMNR,26524000.0


In [29]:
master = master.sort_index(level=0, ascending=True, sort_remaining=True) # sort_remaining says it'll sort by ticker second
master

AR
Dates      Ticker              
2005-03-31 AAP     8.449500e+07
           ABG     1.513290e+08
           ALV     1.352000e+09
           AMZN    1.550000e+08
           AN      6.154000e+08
...                         ...
2021-09-30 WMT     5.770000e+09
           WSM     1.101310e+08
           WW      3.637700e+07
           WWW     3.321000e+08
           ZUMZ    2.481100e+07

[18546 rows x 1 columns]

In [30]:
master.to_excel('AR_Final.xlsx')